In [6]:
%%bash
apt-get install openjdk-8-jdk-headless -qq > /dev/null

wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
tar xf spark-3.0.0-bin-hadoop3.2.tgz

pip install findspark

In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [10]:
df = spark.read.csv('/content/gdrive/MyDrive/DataFolder/brooklyn_sales_map.csv', 
                    inferSchema = True, sep = ',', header = True)

In [11]:
print(df.count(),len(df.columns))

390883 111


In [12]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- borough1: integer (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- building_class_category: string (nullable = true)
 |-- tax_class: string (nullable = true)
 |-- block: integer (nullable = true)
 |-- lot: integer (nullable = true)
 |-- easement: string (nullable = true)
 |-- building_class: string (nullable = true)
 |-- address9: string (nullable = true)
 |-- apartment_number: string (nullable = true)
 |-- zip_code: integer (nullable = true)
 |-- residential_units: integer (nullable = true)
 |-- commercial_units: integer (nullable = true)
 |-- total_units: integer (nullable = true)
 |-- land_sqft: double (nullable = true)
 |-- gross_sqft: double (nullable = true)
 |-- year_built: integer (nullable = true)
 |-- tax_class_at_sale: integer (nullable = true)
 |-- building_class_at_sale: string (nullable = true)
 |-- sale_price: double (nullable = true)
 |-- sale_date: string (nullable = true)
 |-- year_of_sale: integer (nulla

In [13]:
df.show(10)

+---+--------+--------------------+-----------------------+---------+-----+----+--------+--------------+-------------------+----------------+--------+-----------------+----------------+-----------+---------+----------+----------+-----------------+----------------------+------------+----------+------------+---------+---+------+------+----------+-------+-------+--------+----------+----------+----------+---------+----------+--------+-------------------+---------+---------+---------+---------+--------+--------+-------+-------+-------+---------+---------+---------+-------+---------+---------+--------------------+-------+--------+-------+-------+----------+----------+----------+---------+----------+---------+----------+--------+---------+--------+----------+--------+--------+---------+---------+---+--------+----------+-------+--------+----------+---------+----------+---------+---------+----------+----------+--------------------+--------+--------+--------+-------+--------+--------+----------+

In [15]:
dfStore = df.filter(df.building_class_category == '22  STORE BUILDINGS')
dfStore.show(10)

+---+--------+--------------------+-----------------------+---------+-----+---+--------+--------------+--------------------+----------------+--------+-----------------+----------------+-----------+---------+----------+----------+-----------------+----------------------+----------+----------+------------+---------+---+------+------+----------+-------+-------+--------+----------+----------+----------+---------+----------+--------+------------------+---------+---------+---------+---------+--------+--------+-------+-------+-------+---------+---------+---------+-------+---------+---------+--------------------+-------+--------+-------+-------+----------+----------+----------+---------+----------+---------+----------+--------+---------+--------+----------+--------+--------+---------+---------+---+--------+----------+-------+--------+----------+---------+----------+---------+---------+----------+----------+--------+--------+--------+--------+-------+--------+--------+----------+-------+-------

In [16]:
dfStore2015 = dfStore.filter((dfStore.sale_price>30000000)&(dfStore.year_of_sale>=2015))
dfStore2015.show(10)

+---+--------+--------------------+-----------------------+---------+-----+---+--------+--------------+--------------------+----------------+--------+-----------------+----------------+-----------+---------+----------+----------+-----------------+----------------------+----------+----------+------------+---------+---+------+------+----------+-------+-------+--------+----------+----------+----------+---------+----------+--------+-----------------+---------+---------+---------+---------+--------+--------+-------+-------+-------+---------+---------+---------+-------+---------+---------+--------------------+-------+--------+-------+-------+----------+----------+----------+---------+----------+---------+----------+--------+---------+--------+----------+--------+--------+---------+---------+---+--------+----------+-------+--------+----------+---------+----------+---------+---------+----------+----------+--------+--------+--------+--------+-------+--------+--------+----------+-------+--------

In [17]:
dfCategoriesCount = df.groupBy('building_class_category').count()
dfCategoriesCount.show()

+-----------------------+------+
|building_class_category| count|
+-----------------------+------+
|   32  HOSPITAL AND ...|   318|
|         17 CONDO COOPS|    29|
|   49 CONDO WAREHOUS...|    24|
|   11  SPECIAL CONDO...|     4|
|            17  CONDOPS|   448|
|      44  CONDO PARKING|  2596|
|    01 ONE FAMILY HOMES| 54205|
|   31 COMMERCIAL VAC...|   132|
|   48  CONDO TERRACE...|    90|
|   03 THREE FAMILY H...| 38691|
|   16  CONDOS - 2-10...|   439|
|    02 TWO FAMILY HOMES|106127|
|           27 FACTORIES|    84|
|          30 WAREHOUSES|   133|
|                   null|    83|
|   28 COMMERCIAL CONDOS|     7|
|   47 CONDO NON-BUSI...|   100|
|   36 OUTDOOR RECREA...|     4|
|   05  TAX CLASS 1 V...|  7377|
|   15  CONDOS - 2-10...|  8913|
+-----------------------+------+
only showing top 20 rows



In [18]:
dfStoreOrdered = dfStore.select('address9','year_built','year_of_sale',
                                'sale_price').orderBy('year_of_sale',
                                                      'sale_price',
                                                      ascending = [True,
                                                                   False])
dfStoreOrdered.show(15)

+--------------------+----------+------------+----------+
|            address9|year_built|year_of_sale|sale_price|
+--------------------+----------+------------+----------+
|3860-80 NOSTRAND ...|      1959|        2003|    1.75E7|
|  1104 KINGS HIGHWAY|      1971|        2003|     1.3E7|
|    429-39 86 STREET|      1922|        2003| 8000000.0|
|1504 CONEY ISLAND...|         0|        2003| 5625000.0|
|446-48 FULTON STREET|      1925|        2003| 4900000.0|
|      7011 13 AVENUE|      2003|        2003| 4870075.0|
|        435 AVENUE P|      2007|        2003| 4450000.0|
| 847 FLATBUSH AVENUE|      1939|        2003| 4300000.0|
| 847 FLATBUSH AVENUE|      1931|        2003| 4300000.0|
| 847 FLATBUSH AVENUE|      1939|        2003| 4300000.0|
|   2343 UTICA AVENUE|         0|        2003| 3800000.0|
|237-43 DUFFIELD S...|         0|        2003| 3675000.0|
|    1769 BATH AVENUE|      1931|        2003| 3307131.0|
|   8704-08 18 AVENUE|      1931|        2003| 3307131.0|
|   423 FULTON